In [1]:
import pandas as pd
from rdkit.Chem import MolFromSmiles, MolToSmiles

import mols2grid
from rdkit.Chem.MolStandardize import rdMolStandardize

import rdkit
print(rdkit.__version__)

2020.03.3


Read Molecules

In [3]:
df = pd.read_csv('small_test.csv')
mols = [MolFromSmiles(smi) for smi in df['smiles']]

mols2grid.display(mols,
                  tooltip=["SMILES"])

### Call Tautomer Enumerator

In [ ]:
n=0
enumerator = rdMolStandardize.TautomerEnumerator()


tauts = enumerator.Enumerate(mols[n])
canon = enumerator.Canonicalize(mols[n])
tauts = [MolFromSmiles(MolToSmiles(x)) for x in tauts]

order so that canonical tautomer is always first

In [27]:
def orderedTautomers(m, enumerator):
    enumerator = rdMolStandardize.TautomerEnumerator()
    canon = enumerator.Canonicalize(m)
    csmi = MolToSmiles(canon)
    res = [canon]
    tauts = enumerator.Enumerate(m)
    smis = [MolToSmiles(x) for x in tauts]
    stpl = sorted((x,y) for x,y in zip(smis,tauts) if x!=csmi)
    res += [y for x,y in stpl]
    return res

n=6
mols2grid.display(orderedTautomers(mols[n], enumerator),
                  tooltip=["SMILES"])

In [ ]:
from tqdm import tqdm

enumerator = rdMolStandardize.TautomerEnumerator()
def return_taut(smi):
    canon = enumerator.Canonicalize(MolFromSmiles(smi))
    return MolToSmiles(canon)

df = pd.read_csv('ugis_processed_new.csv')

tqdm.pandas()
df['canon'] = df['smiles'].progress_apply(return_taut)
df['smiles'] = df['canon']
df.reindex(columns=['smiles', 'dock_score'])
df.to_csv('ugis_canon.csv')

In [3]:
mpi_size=448
file_dir = '/rds-d2/user/wjm41/hpc-work/datasets/Ugis/scripts'
dfs = [pd.read_csv(file_dir+'/tmp/tautcan_'+str(n)+'.csv') for n in range(mpi_size)]
df = pd.concat(dfs)
df.to_csv(file_dir+'/tautcan_lib.csv', index=False)


In [5]:
print(len(df))
df_og = pd.read_csv('ugis_processed_new.csv')
print(len(df_og))
assert len(df) == len(df_og)


7503221
7503221
